In [2]:
import yt_dlp
import whisper
import re

In [ ]:
def download_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': '%(id)s.%(ext)s',
        'ffmpeg_location': '/opt/homebrew/bin/ffmpeg',  # Update with your ffmpeg path
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)
    return result['id'] + '.mp3'

def clean_transcription(text):
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)
    
    # Capitalize first letter of each sentence
    text = '. '.join(sentence.capitalize() for sentence in text.split('. '))
    
    # Remove any repeated words
    text = re.sub(r'\b(\w+)(\s+\1\b)+', r'\1', text)
    
    return text.strip()

def transcribe_and_save(url, output_filename='profg.txt'):
    # Download audio
    audio_file = download_audio(url)
    
    # Transcribe
    model = whisper.load_model("base")
    result = model.transcribe(audio_file)
    
    # Clean transcription
    cleaned_text = clean_transcription(result['text'])
    
    # Save to text file
    with open(output_filename, 'w', encoding='utf-8') as file:
        file.write(cleaned_text)
    
    print(f"Transcription saved to {output_filename}")
    return cleaned_text

# Example usage
video_url = "https://www.youtube.com/watch?v=J7d1qC6D7JM&ab_channel=TheProfGShow%E2%80%93ScottGalloway" #Replace this with your Youtube link
transcription = transcribe_and_save(video_url)
print(transcription)